In [4]:
!pip install transformers --upgrade
!pip install datasets
!pip install trl[peft] --upgrade
!pip install -U git+https://github.com/huggingface/trl
!pip install bitsandbytes loralib
!pip install wandb -U
!pip install hf_transfer
!pip install sentencepiece



[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip
  Cloning https://github.com/huggingface/trl to /tmp/pip-req-build-_ncebobc
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl /tmp/pip-req-build-_ncebobc
  Resolved https://github.com/huggingface/trl to commit 5828a666bff52eb18c1107317c0dfb54f57430b8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for trl: filename=trl-0.9.6.dev0-py3-none-any.whl size=246899 sha256=70e9327d1f5fbdec79ee6f8485308fa360ac81f91f39661f059da25a877807c9
  Stored in directory: /tmp/pip-ephem-wheel-cache-_jy_1

In [5]:
%env HF_HUB_ENABLE_HF_TRANSFER=True
%env WANDB_PROJECT=LLM-Training-Course
%env WANDB_RUN_ID=DORA
%env WANDB_NOTEBOOK_NAME={__vsc_ipynb_file__}

env: HF_HUB_ENABLE_HF_TRANSFER=True
env: WANDB_PROJECT=LLM-Training-Course
env: WANDB_RUN_ID=DORA
env: WANDB_NOTEBOOK_NAME=/root/llm-training-course/03-sft-and-training-tricks/04-train-with-dora.ipynb


In [6]:
import wandb
wandb.login()

wandb: Currently logged in as: gal-apriori. Use `wandb login --relogin` to force relogin


True

In [7]:
import sys
sys.path.append('/root/llm-training-course/')

In [8]:
model_id = "mistralai/Mistral-7B-Instruct-v0.3"

In [9]:
from datasets import load_dataset
train_ds, eval_ds = load_dataset("mlabonne/orpo-dpo-mix-40k", split=["train[:10%]","train[10%:15%]"])

In [10]:
train_ds

Dataset({
    features: ['source', 'chosen', 'rejected', 'prompt', 'question'],
    num_rows: 4424
})

In [11]:
train_ds = train_ds.map(lambda x: { "messages": [{"role":"user", "content": x["prompt"] + "\n" +  x["chosen"][0]["content"]}] + x["chosen"][1:] })
eval_ds = eval_ds.map(lambda x: { "messages": [{"role":"user", "content": x["prompt"]+ "\n"+ x["chosen"][0]["content"]}]+ x["chosen"][1:] })

In [12]:
columns_to_remove = [c for c in train_ds.column_names if c not in ["messages"]]
train_ds = train_ds.remove_columns(columns_to_remove)

columns_to_remove = [c for c in eval_ds.column_names if c not in ["messages"]]
eval_ds = eval_ds.remove_columns(columns_to_remove)

In [13]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.12it/s]


In [14]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

In [15]:
tokenizer.chat_template

"{{bos_token}}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if message['role'] == 'user' %}{{ '[INST] ' + message['content'] + '[/INST]' }}{% elif message['role'] == 'assistant' %}{{ ' ' + message['content'] + eos_token}}{% else %}{{ raise_exception('Only user and assistant roles are supported!') }}{% endif %}{% endfor %}"

In [16]:
from helpers import set_padding_for_tokenizer
set_padding_for_tokenizer(tokenizer)

In [17]:
from helpers import stream_responses_for_sample
from transformers import GenerationConfig

generation_config =  GenerationConfig(max_new_tokens=250, eos_token_id=[tokenizer.eos_token_id])
sample_conversations = [
    [{"role": "user", "content": "What is the capital of France?"}],
    [{"role": "user", "content": "Write me a javascript function that check if string is palindrome."}],
    [{"role": "user", "content": "Given x^2=36-4 what is x?"}]
]
stream_responses_for_sample(model, tokenizer, sample_conversations, generation_config=generation_config)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] What is the capital of France?[/INST] The capital of France is Paris. It is one of the most famous cities in the world, known for its rich history, art, culture, and landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. Paris is also the political, economic, and cultural center of France.</s>


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] Write me a javascript function that check if string is palindrome.[/INST] Here is a simple JavaScript function that checks if a string is a palindrome. This function will ignore spaces, punctuation, and convert the string to lowercase for comparison.

```javascript
function isPalindrome(str) {
   const re = /[\W_]/g;
   let cleanStr = str.toLowerCase().replace(re, '');
   let reversedStr = cleanStr.split('').reverse().join('');

   return cleanStr === reversedStr;
}
```

You can use this function like this:

```javascript
console.log(isPalindrome("A man, a plan, a canal: Panama")); // true
console.log(isPalindrome("Not a palindrome")); // false
```

This function will return `true` if the input string is a palindrome and `false` otherwise.</s>


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] Given x^2=36-4 what is x?[/INST] To find the value of x, you first need to rearrange the equation to isolate x. Here's how:

1. Add 4 to both sides of the equation:

x^2 + 4 = 36

2. Then, subtract 4 from both sides:

x^2 + 4 - 4 = 36 - 4

3. This simplifies to:

x^2 = 32

4. Now, take the square root of both sides. Remember that when you take the square root, you get two solutions, one positive and one negative:

x = ±√32

5. Calculate the square root of 32:

x = ±4√3

So, the solutions for x are x = 4√3 and x = -4√3.</s>


In [18]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

In [25]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                     "gate_proj", "up_proj","down_proj"],
    modules_to_save=["embed_tokens", "input_layernorm", "post_attention_layernorm"],
    bias="none",
    task_type="CAUSAL_LM",
    use_dora=True
)


In [26]:
from peft import get_peft_model
peft_model = get_peft_model(model, peft_config)

In [27]:
peft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): ModulesToSaveWrapper(
          (original_module): Embedding(32768, 4096)
          (modules_to_save): ModuleDict(
            (default): Embedding(32768, 4096)
          )
        )
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): Para

In [30]:
peft_model.base_model.model.model.layers[0].self_attn.q_proj.lora_magnitude_vector.training

True

In [31]:
import os
from trl import SFTConfig, SFTTrainer

args = SFTConfig(
    output_dir=os.getenv("WANDB_RUN_ID"),
    report_to="wandb",
    num_train_epochs=1.0,
    do_train=True,
    do_eval=True,
    log_level="debug",
    gradient_checkpointing=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    per_device_eval_batch_size=1,
    lr_scheduler_type="constant",
    bf16=True,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=0.1,
    max_grad_norm=.3,
    learning_rate=1e-4
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [32]:
trainer = SFTTrainer(
    model=peft_model,
    tokenizer=tokenizer,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:289: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:366: UserWarning: You passed a `dataset_kwargs` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:408: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
You have loaded a model on multiple GPUs. `is_model_parallel` attribute will be force-set to `True` to avoid any unexpected behavior such as device placement mismatching.
Using auto half precision backend
Currently tr

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
56,0.906000,0.988357
112,0.890800,0.980615
168,0.896200,0.982692
224,0.885700,0.984340



***** Running Evaluation *****
  Num examples = 2213
  Batch size = 1
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)

***** Running Evaluation *****
  Num examples = 2213
  Batch size = 1

***** Running Evaluation *****
  Num examples = 2213
  Batch size = 1

***** Running Evaluation *****
  Num examples = 2213
  Batch size = 1
Saving model checkpoint to DORA/checkpoint-276
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.3/snapshots/0417f4babd26db0b5ed07c1d0bc85658ab526ea3/config.json
Model config MistralConfig {
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,


TrainOutput(global_step=276, training_loss=0.9049716825070588, metrics={'train_runtime': 11624.5028, 'train_samples_per_second': 0.381, 'train_steps_per_second': 0.024, 'total_flos': 2.247993653162066e+17, 'train_loss': 0.9049716825070588, 'epoch': 0.9981916817359855})